In [1]:
using SciBmad
using DifferentiationInterface
import DifferentiationInterface as DI
# Different differentiation packages:
using ForwardDiff, GTPSA, ReverseDiff, Mooncake, FiniteDiff, FiniteDifferences

In [2]:
K1 = 0.36
K2 = 0.1

qf = Quadrupole(L=0.5, Kn1=K1)
sf = Sextupole(L=0.2, Kn2=K2)
d = Drift(L=0.6)
b = SBend(L=6.0, angle=π/132)
qd = Quadrupole(L=0.5, Kn1=-K1)
sd = Sextupole(L=0.2, Kn2=-K2)
fodo = Beamline([qf, d, sf, b, sd, d, qd], species_ref=Species("electron"), E_ref=18e9);

In [16]:
# Returns exit coordinates for 1 particle
function track_a_particle(v0)
    v0_ = similar(v0)
    v0_ .= v0
    b0 = Bunch(v0_; species=fodo.species_ref, R_ref=fodo.R_ref)
    track!(b0, fodo)
    return b0.coords.v
end
x0 = [1e-2, 0.0, 1e-2, 0.0, 0.0, 0.0];

In [13]:
# Forward AD using ForwardDiff.jl:
prep_FD = DI.prepare_jacobian(track_a_particle, AutoForwardDiff(), x0, Constant(fodo))
J_FD = DI.jacobian(track_a_particle, prep_FD, AutoForwardDiff(), x0, Constant(fodo))

6×6 Matrix{Float64}:
 -0.501301     8.58617      0.000765453  -0.00240033  0.0   0.122029
 -0.257394     2.41377     -0.000629133  -0.00359376  0.0   0.0439702
  0.00154156  -0.00171283   2.41678       8.59153     0.0  -0.013781
  4.81099e-5  -0.00281752  -0.256971     -0.499744    0.0   0.00253851
 -0.00937178  -0.0829428   -0.0027041    -0.0152556   1.0  -0.000483029
  0.0          0.0          0.0           0.0         0.0   1.0

In [14]:
# Taylor AD using GTPSA.jl:
prep_GTPSA = DI.prepare_jacobian(track_a_particle, AutoGTPSA(), x0, Constant(fodo))
J_GTPSA = DI.jacobian(track_a_particle, prep_GTPSA, AutoGTPSA(), x0, Constant(fodo))

6×6 Matrix{Float64}:
 -0.501301     8.58617      0.000765453  -0.00240033  0.0   0.122029
 -0.257394     2.41377     -0.000629133  -0.00359376  0.0   0.0439702
  0.00154156  -0.00171283   2.41678       8.59153     0.0  -0.013781
  4.81099e-5  -0.00281752  -0.256971     -0.499744    0.0   0.00253851
 -0.00937178  -0.0829428   -0.0027041    -0.0152556   1.0  -0.000483029
  0.0          0.0          0.0           0.0         0.0   1.0

In [15]:
# Reverse AD using ReverseDiff.jl:
prep_RD = DI.prepare_jacobian(track_a_particle, AutoReverseDiff(;compile=true), x0, Constant(fodo))
J_RD = DI.jacobian(track_a_particle, prep_RD, AutoReverseDiff(;compile=true), x0, Constant(fodo))

LoadError: ArgumentError: Implement `KernelAbstractions.get_backend(::ReverseDiff.TrackedArray{Float64, Float64, 2, Matrix{Float64}, Matrix{Float64}})`

In [7]:
# Reverse AD using Mooncake.jl:
prep_MR = DI.prepare_jacobian(track_a_particle, AutoMooncake(), x0)
J_MR = DI.jacobian(track_a_particle, prep_MR, AutoMooncake(), x0)

6×6 Matrix{Float64}:
 -0.501301     8.58617      0.000765453  -0.00240033  0.0   0.122029
 -0.257394     2.41377     -0.000629133  -0.00359376  0.0   0.0439702
  0.00154156  -0.00171283   2.41678       8.59153     0.0  -0.013781
  4.81099e-5  -0.00281752  -0.256971     -0.499744    0.0   0.00253851
 -0.00937178  -0.0829428   -0.0027041    -0.0152556   1.0  -0.000483029
  0.0          0.0          0.0           0.0         0.0   1.0

In [8]:
# Finite differencing using FiniteDiff.jl:
prep_fdif1 = DI.prepare_jacobian(track_a_particle, AutoFiniteDiff(), x0)
J_fdif1 = DI.jacobian(track_a_particle, prep_fdif1, AutoFiniteDiff(), x0)

6×6 Matrix{Float64}:
 -0.501301     8.58617      0.000765455  -0.00240034  0.0   0.122029
 -0.257394     2.41377     -0.000629133  -0.00359377  0.0   0.0439702
  0.00154156  -0.00171282   2.41678       8.59153     0.0  -0.013781
  4.81099e-5  -0.00281752  -0.256971     -0.499744    0.0   0.00253851
 -0.00937178  -0.0829429   -0.00270414   -0.0152558   1.0  -0.000483002
  0.0          0.0          0.0           0.0         0.0   1.0

In [9]:
# Finite differencing using FiniteDifferences.jl:
prep_fdif2 = DI.prepare_jacobian(track_a_particle, AutoFiniteDifferences(;fdm=central_fdm(3,1)), x0)
J_fdif2 = DI.jacobian(track_a_particle, prep_fdif2, AutoFiniteDifferences(;fdm=central_fdm(3,1)), x0)

6×6 Matrix{Float64}:
 -0.501301     8.58617      0.000765453  -0.00240033  0.0   0.122029
 -0.257394     2.41377     -0.000629133  -0.00359376  0.0   0.0439702
  0.00154156  -0.00171283   2.41678       8.59153     0.0  -0.013781
  4.81099e-5  -0.00281752  -0.256971     -0.499744    0.0   0.00253851
 -0.00937178  -0.0829428   -0.0027041    -0.0152556   1.0  -0.000483029
  0.0          0.0          0.0           0.0         0.0   1.0

In [10]:
# Show they're all equal:
J_FD ≈ J_GTPSA ≈ J_RD ≈ J_MR ≈ J_fdif1 ≈ J_fdif2

true